In [1]:
from pyo import *
import re

In [2]:
class MyInstrument(EventInstrument):
    def __init__(self, **args):
        EventInstrument.__init__(self, **args)
        
        self.env = Adsr(0.01, 0.05, 0.5, 1, dur=self.dur+1, mul=self.amp).play()

        # self.freq is derived from the 'degree' argument.
        self.phase = Phasor([self.freq, self.freq * 1.003])

        # self.dur is derived from the 'beat' argument.
        self.duty = Expseg([(0, 0.05), (self.dur, 0.5)], exp=4).play()

        self.osc = Compare(self.phase, self.duty, mode="<", mul=1, add=-0.5)

        # EventInstrument created the amplitude envelope as self.env.
        self.filt = ButLP(self.osc, freq=5000, mul=self.env).out()

In [3]:
def sep_abc(s):
    s = s.replace(' ', '')
    s = s.replace('|', '')
    return re.findall(r"[_^]?[A-G,a-g][,']*\d?/?\d*", s)
ar = sep_abc("CEG")
ar

['C', 'E', 'G']

In [4]:
def note2midi(s):
    p = r"([_^]?)([A-G,a-g])([,']*)(\d?/?\d*)"
    r = re.match(p, s)
    sign = r.group(1)
    note = r.group(2)
    register = r.group(3)
    dur = r.group(4)

    # sign
    sign_shift = 0
    if sign =='^': sign_shift = 1
    if sign == '_': sign_shift = -1

    # note
    if note.isupper():
        oct = 0
    else:
        oct = 12

    dic = {'c': 48, 'd': 50, 'e': 52, 'f': 53, 'g': 55, 'a': 57, 'b': 59}

    # octave
    if register == "'" : oct = oct + 12
    if register == "''": oct = oct + 24
    if register == "," : oct = oct - 12
    if register == ",,": oct = oct - 24
        
    # dur
    if dur=='': dur = "1"
    if dur.startswith('/'):
        dur = "1" + dur
    dur = eval(dur)
    
    return dic[note.lower()] + sign_shift + oct, dur

In [5]:
def abc(s="CEGc", instr=None, amp=0.1, occ=1, beat=1):
    if instr is None:
        instr = DefaultInstrument
    ar = sep_abc(s)
    midinote = [note2midi(s)[0] for s in ar]
    dur = [note2midi(s)[1] for s in ar]
    e = Events(amp = amp, beat=beat, instr=instr, midinote = EventSeq(midinote, occ), dur = EventSeq(dur, occ))
    return e

In [6]:
s = Server().boot().start()

In [8]:
a = abc().play();

In [9]:
p1 = abc(amp=0.01, beat=0.5, instr=MyInstrument, s="e'/4d'/4^f/2^g/2       | ^c'/4b/4d/2e/2 b/4   | a/4 ^c/2 e/2 a3/2   ").play()
p2 = abc(amp=0.01, beat=0.5, instr=MyInstrument, s="B,/4B/4B,/4B,/4B/4B,/4 | E,/4E/4E,/4E,/4E/4E, | A,/4A/4A,/4").play()

In [10]:
s.stop()